In [3]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en drive
import os
print(os.getcwd())
os.chdir("/content/gdrive/My Drive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [4]:
!pip install folium
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%cd '/content/gdrive/MyDrive/Curso Data Science para todos/Datos/'

/content/gdrive/MyDrive/Curso Data Science para todos/Datos


**Fuentes de datos**

En los siguientes enlaces encontraran los datos en formato GEOJSON y el csv para este caso

1. [GEOJSON](https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code&disjunctive.ste_name&disjunctive.coty_code&disjunctive.coty_name&sort=year)
2. [Data COVID](https://data.cdc.gov/Public-Health-Surveillance/United-States-COVID-19-County-Level-of-Community-T/8396-v7yb/data)

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip

#Leer el archivo geoJSON
geojson = gpd.read_file('georef-united-states-of-america-county.geojson')
geojson=geojson[['coty_code','geometry']] #seleccionar 'coty_code' (country fips) y 'geometry' 

#Covid data
covid_df=pd.read_csv('United_States_COVID-19_County_Level_of_Community_Transmission_as_Originally_Posted.csv')

In [7]:
geojson

,coty_code,geometry
0,01119,"POLYGON ((-87.85342 32.53209, -87.85961 32.532..."
1,02230,"POLYGON ((-135.03053 59.34635, -134.98974 59.3..."
2,05011,"POLYGON ((-92.35028 33.29884, -92.35226 33.293..."
3,05069,"POLYGON ((-92.03025 34.48954, -92.08334 34.490..."
4,05085,"POLYGON ((-92.12154 35.01220, -92.07081 35.011..."
...,...,...
3229,55033,"POLYGON ((-92.13520 44.68437, -92.02842 44.683..."
3230,55057,"POLYGON ((-89.78581 43.64105, -89.78490 43.641..."
3231,72011,"POLYGON ((-67.08200 18.25495, -67.07891 18.258..."
3232,72043,"POLYGON ((-66.33124 18.01589, -66.32574 18.018..."


In [13]:
covid_df.head(15)

,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
0,Michigan,Ontonagon County,26131,2021/08/16,suppressed,4.44,substantial
1,South Dakota,Corson County,46031,2021/08/16,244.740,30.43,high
2,Kentucky,Bracken County,21023,2021/08/16,397.450,9.93,high
3,Texas,Kleberg County,48273,2021/08/16,576.920,11.06,high
4,Iowa,Winnebago County,19189,2021/08/16,164.190,13.76,high
5,Missouri,Mississippi County,29133,2021/08/16,493.170,32.89,high
6,Pennsylvania,Crawford County,42039,2021/08/16,73.260,7.88,substantial
7,Virginia,Nelson County,51125,2021/08/16,113.860,7.44,high
8,Iowa,Poweshiek County,19157,2021/08/16,70.260,9.39,substantial
9,Michigan,Emmet County,26047,2021/08/16,86.790,7.62,substantial


In [9]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098442 entries, 0 to 1098441
Data columns (total 7 columns):
 #   Column                                              Non-Null Count    Dtype  
---  ------                                              --------------    -----  
 0   state_name                                          1098442 non-null  object 
 1   county_name                                         1098442 non-null  object 
 2   fips_code                                           1098442 non-null  int64  
 3   report_date                                         1098442 non-null  object 
 4   cases_per_100K_7_day_count_change                   1095953 non-null  object 
 5   percent_test_results_reported_positive_last_7_days  955238 non-null   float64
 6   community_transmission_level                        1097359 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 58.7+ MB


In [10]:
covid_df.report_date.min()

'2021/08/16'

In [11]:
covid_df.report_date.max()

'2022/07/23'

Este conjunto de datos realiza un seguimiento de las métricas de Covid-19 a nivel de condado de EE. UU. para cada report_date. Dado que estamos creando un mapa Cloropeth estático, limitaremos nuestro marco de datos a la fecha más reciente (es decir, 23/07/2022 en nuestro ejemplo)

También necesitamos hacer un poco de Data wrangling para limpiar algunos de los campos de datos. Cambiaremos el tipo de datos de 'fips_code' a una cadena y completaremos el cero inicial si falta. También crearemos dos nuevas columnas 'new_cases_7days' y 'pct_positive_7days' 

In [14]:
#cambiar tipo de data 'fips_code' a string y llenar con zeros
covid_df["fips_code"]=covid_df["fips_code"].map(str)
covid_df["fips_code"] = covid_df["fips_code"].str.zfill(5)

In [16]:
covid_df.head(15)

,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
0,Michigan,Ontonagon County,26131,2021/08/16,suppressed,4.44,substantial
1,South Dakota,Corson County,46031,2021/08/16,244.740,30.43,high
2,Kentucky,Bracken County,21023,2021/08/16,397.450,9.93,high
3,Texas,Kleberg County,48273,2021/08/16,576.920,11.06,high
4,Iowa,Winnebago County,19189,2021/08/16,164.190,13.76,high
5,Missouri,Mississippi County,29133,2021/08/16,493.170,32.89,high
6,Pennsylvania,Crawford County,42039,2021/08/16,73.260,7.88,substantial
7,Virginia,Nelson County,51125,2021/08/16,113.860,7.44,high
8,Iowa,Poweshiek County,19157,2021/08/16,70.260,9.39,substantial
9,Michigan,Emmet County,26047,2021/08/16,86.790,7.62,substantial


In [17]:
#Filtrar la fecha de interes
covid_df=covid_df[(covid_df['report_date'] =='2022/07/23')]
covid_df.head()

,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
1095220,Alabama,Bullock County,01011,2022/07/23,415.800,42.86,high
1095221,Alabama,Choctaw County,01023,2022/07/23,suppressed,6.25,substantial
1095222,Alabama,Clarke County,01025,2022/07/23,609.600,43.56,high
1095223,Alabama,Colbert County,01033,2022/07/23,521.350,31.92,high
1095224,Alabama,Covington County,01039,2022/07/23,350.890,27.87,high


In [18]:
#crear dos columnas nuevas con tipos de datos correctos y borrar las originales
import warnings
warnings.filterwarnings("ignore")
covid_df['new_cases_7days'] = covid_df['cases_per_100K_7_day_count_change'].str.replace(',', '')
covid_df['new_cases_7days'] = covid_df['new_cases_7days'].replace({'suppressed': np.nan}).astype(float)
covid_df['pct_positive_7days']=covid_df['percent_test_results_reported_positive_last_7_days']/100
covid_df.drop(['cases_per_100K_7_day_count_change', 'percent_test_results_reported_positive_last_7_days',
              'community_transmission_level'], axis=1, inplace=True)

In [19]:
covid_df.head()

,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
1095220,Alabama,Bullock County,01011,2022/07/23,415.80,0.4286
1095221,Alabama,Choctaw County,01023,2022/07/23,NaN,0.0625
1095222,Alabama,Clarke County,01025,2022/07/23,609.60,0.4356
1095223,Alabama,Colbert County,01033,2022/07/23,521.35,0.3192
1095224,Alabama,Covington County,01039,2022/07/23,350.89,0.2787


In [20]:
#Join the geojson file con covid_df
df_final = geojson.merge(covid_df, left_on="coty_code", right_on="fips_code", how="outer") 
df_final = df_final[~df_final['geometry'].isna()]
df_final.head()

,coty_code,geometry,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,01119,"POLYGON ((-87.85342 32.53209, -87.85961 32.532...",Alabama,Sumter County,01119,2022/07/23,297.74,0.2222
1,02230,"POLYGON ((-135.03053 59.34635, -134.98974 59.3...",Alaska,Skagway Muny,02230,2022/07/23,NaN,0.2308
2,05011,"POLYGON ((-92.35028 33.29884, -92.35226 33.293...",Arkansas,Bradley County,05011,2022/07/23,250.86,0.0695
3,05069,"POLYGON ((-92.03025 34.48954, -92.08334 34.490...",Arkansas,Jefferson County,05069,2022/07/23,299.29,0.1148
4,05085,"POLYGON ((-92.12154 35.01220, -92.07081 35.011...",Arkansas,Lonoke County,05085,2022/07/23,372.40,0.1598


# Crear mapa base

In [21]:
m = folium.Map(location=[40, -96], zoom_start=4,tiles='openstreetmap')
m

# Adicionar objeto cloropeth

In [22]:
# escala de colores
custom_scale = (df_final['new_cases_7days'].quantile((0,0.2,0.95,1))).tolist()
custom_scale=[round(x,0) for x in custom_scale]
custom_scale

[0.0, 154.0, 505.0, 2328.0]

Algunos parametros importantes a la hora de adicionar el objeto cloropeth

1. geo_data = path del archivo GEOJSON
2. data = dataframe que contiene los datos
3. columns = las dos columnas que se usaran apra crear el mapa cloropeth, la primera es la identificadora de la ubicacion geografica (e.g nombre pais, zip code, etc) y la segunda es la metrica que representa el area de sombreado en el mapa
4. key_on= llave en el archivo GEOJSON que es la identificadora de las propiedades que se usa para hacer el match con las ubicaciones del dataframe con la data en este ejemplo es la columna *coty_code*

In [24]:
folium.Choropleth(
            geo_data='georef-united-states-of-america-county.geojson',
            data=df_final,
            columns=['fips_code', 'new_cases_7days'],  #Aquí le decimos a folium que obtenga los fips del condado y trace la métrica new_cases_7days para cada condado
            key_on='feature.properties.coty_code', #Aquí tomamos las geometrías/límites del condado del archivo geojson usando la clave 'coty_code' que es lo mismo que los fips del condado
            threshold_scale=custom_scale, #use la escala personalizada que creamos para la leyenda
            fill_color='YlOrRd',
            nan_fill_color="White", #Utilice el color blanco si no hay datos disponibles para el condado
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Casos nuevos en los ultimos 7 Dias (Por 100K Poblacion) ', #titulo de leyenda
            highlight=True,
            line_color='black').add_to(m) 

m

Output hidden; open in https://colab.research.google.com to view.

# Agregar elementos adicionales

In [25]:
folium.features.GeoJson?

In [26]:
#Edicion avanzada
m = folium.Map(location=[40, -96], zoom_start=4,tiles='openstreetmap')
folium.Choropleth(
            geo_data='georef-united-states-of-america-county.geojson',
            data=df_final,
            columns=['fips_code', 'new_cases_7days'],  #Aquí le decimos a folium que obtenga los fips del condado y trace la métrica new_cases_7days para cada condado
            key_on='feature.properties.coty_code', #Aquí tomamos las geometrías/límites del condado del archivo geojson usando la clave 'coty_code' que es lo mismo que los fips del condado
            threshold_scale=custom_scale, #use la escala personalizada que creamos para la leyenda
            fill_color='YlOrRd',
            nan_fill_color="White", #Utilice el color blanco si no hay datos disponibles para el condado
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Casos nuevos en los ultimos 7 Dias (Por 100K Poblacion) ', #titulo de leyenda
            highlight=True,
            line_color='black').add_to(m)

folium.features.GeoJson(
                    data=df_final,
                    name='Nuevos casos en ultimos 7 dias (Por 100K Poblacion)',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['report_date',
                                'county_name',
                                'state_name',
                                'new_cases_7days',
                                'pct_positive_7days'
                               ],
                        aliases=["Fecha reporte:",
                                 "Nombre condado:",
                                 "Nombre estado:",
                                 "Nuevos casos utlimos 7 dias<br>(Por 100K Poblacion):",
                                 "Porcentaje de casos + <br>Ultimos 7días:"
                                ], 
                      )).add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.